# pandas的两个主要数据结构：`Series`和`DataFrame`。

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

# Series
## 创建Series    `pd.Series(list/array)`
* Series是一种类似于一维数组的对象，它由一组（类似NumPy数据类型的）数据以及一组与之相关的数据标签（即索引）组成。仅由一个数组即可创建最简单的Series。  
* Series字符串以交互式的方式呈现，索引位于左边，值位于右边。如果没有为数据指定索引，则会自动创建一个0到N-1（N为数据的长度）的整数型索引。

In [2]:
obj = pd.Series([4, 7, -5, 3])

In [3]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

## 获取Series数组值`.array`和索引对象`.index`

In [4]:
obj.array

<NumpyExtensionArray>
[np.int64(4), np.int64(7), np.int64(-5), np.int64(3)]
Length: 4, dtype: int64

In [5]:
obj.index

RangeIndex(start=0, stop=4, step=1)

## 创建带有索引的Series    `pd.Series(list/array, index=list)`

In [6]:
obj2 = pd.Series([4, 7, -5, 3], index=["d", "b", "a", "c"])

In [7]:
obj2

d    4
b    7
a   -5
c    3
dtype: int64

## 通过索引的标签选取Series中的单个或一组值    `obj2[str]` 、 `obj2[[str, str, ...]]`

In [8]:
obj2["a"]

np.int64(-5)

In [9]:
obj2["d"] = 6

In [10]:
obj2[["c", "a", "d"]]

c    3
a   -5
d    6
dtype: int64

In [11]:
obj2

d    6
b    7
a   -5
c    3
dtype: int64

## 使用NumPy函数或类似NumPy的运算，比如用布尔型数组进行过滤、标量乘法、应用数学函数等，都会保留索引值的链接。

In [12]:
obj2 > 0

d     True
b     True
a    False
c     True
dtype: bool

In [13]:
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

In [14]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [15]:
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

## 还可以将Series看作长度固定的有序字典，因为它是索引值对数据值的映射。在可能使用字典的场景中，也可以使用Series。

In [16]:
"b" in obj2

True

In [17]:
"e" in obj2

False

## 如果数据已经存放在一个Python字典中，可以通过传入这个字典来创建Series。    `pd.Series(dict)`

In [18]:
sdata = {"Ohio": 35000, "Texas": 71000, "Oregon": 16000, "Utah": 5000}

In [19]:
obj3 = pd.Series(sdata)

In [20]:
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

## 通过to_dict方法，Series也可以转换回字典

In [21]:
obj3.to_dict()

{'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}

## 改变Series的索引顺序。
* 如果只传入一个字典，则结果Series中的索引会遵循字典的键的顺序，而键的顺序依据的是字典的keys方法，keys方法又取决于键的插入顺序。可以将字典键按照想要的顺序传给构造函数，从而使生成的Series的索引顺序符合预期：<br>
* 在下面的例子中，sdata中跟states索引相匹配的三个值放到了相应的位置上，但由于"California"没有对应的值，所以其结果为NaN（即not a number，非数字），在pandas中，它用于标记缺失值或NA值。因为"Utah"不在states中，所以它会从结果中除去。

In [22]:
states = ["California", "Ohio", "Oregon", "Texas"]

In [23]:
obj4 = pd.Series(sdata, index=states)

In [24]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

## 检测缺失数据    `pd.isna(obj4)`、`pd.notna(obj4)`、`obj4.isna()`

In [25]:
pd.isna(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [26]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [27]:
pd.notna(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [28]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [29]:
obj4.isna()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

## Series最实用的一个功能是它在算数运算中能自动对齐索引标签。

In [30]:
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [31]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [32]:
obj4 + obj3

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

## Series对象本身及其索引都有`name`属性，该属性与pandas其他功能的关系非常密切。

In [33]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [34]:
obj4.name = "population"

In [35]:
obj4.index.name = "state"

In [36]:
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

## Series的索引可以直接通过赋值的方式就地修改。

In [37]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [38]:
obj.index = ["Bob", "Steve", "Jeff", "Ryan"]

In [39]:
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

---
# DataFrame
## 创建DataFrame    `pd.DataFrame(等长列表/NumPy数组构成的字典)`
* DataFrame是矩形的数据表，它含有一组有序且有命名的列，每一列可以是不同的数据类型（数值、字符串、布尔值等）。DataFrame既有行索引也有列索引，可以看作由共用同一个索引的Series组成的字典。  
* 有多种创建DataFrame的方式，最常用的是传入一个由等长列表或NumPy数组构成的字典  
* 生成的DataFrame会自动加上索引，跟Series一样，且全部列会按照data键（键的顺序取决于在字典中插入的顺序）的顺序有序排列

In [40]:
data = {"state": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
        "year": [2000, 2001, 2002, 2001, 2002, 2003],
        "pop": [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}

In [41]:
frame = pd.DataFrame(data)

In [42]:
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


In [43]:
pd.DataFrame(np.arange(12).reshape((3,4)))

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


## 对于特别大的DataFrame，`head`方法会只选取前5行，相似地，`tail`方法会返回最后5行。

In [44]:
frame.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


In [45]:
frame.tail()

,state,year,pop
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


## 对DataFrame的列按照指定顺序进行排列。    `pd.DataFrame(dict, columns=list)`

In [46]:
pd.DataFrame(data, columns=["year", "state", "pop"])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


## 如果字典不包含传入的列，就会在结果中产生缺失值。

In [47]:
frame2 = pd.DataFrame(data, columns=["year", "state", "pop", "debt"])

In [48]:
frame2

,year,state,pop,debt
0,2000,Ohio,1.5,NaN
1,2001,Ohio,1.7,NaN
2,2002,Ohio,3.6,NaN
3,2001,Nevada,2.4,NaN
4,2002,Nevada,2.9,NaN
5,2003,Nevada,3.2,NaN


* ## 类似字典标记的方式或点属性的方式，可以将DataFrame的列获取为一个Series。     `DataFrame["column"]`、`DataFrame.column`
* ## `frame2[column]`适用于任意列的名，但是`frame2.column`只有在列名是合理的Python变量名时才适用，并且不能与任意的DataFrame的方法名冲突。例如，如果列名包含空格或下划线以外的符号，就不能用点属性的方式访问。

In [49]:
frame2["state"]

0      Ohio
1      Ohio
2      Ohio
3    Nevada
4    Nevada
5    Nevada
Name: state, dtype: object

In [50]:
frame2.year

0    2000
1    2001
2    2002
3    2001
4    2002
5    2003
Name: year, dtype: int64

## 通过`.iloc`和`.loc`属性即位置或名称的方式进行获取行。

In [51]:
frame2

,year,state,pop,debt
0,2000,Ohio,1.5,NaN
1,2001,Ohio,1.7,NaN
2,2002,Ohio,3.6,NaN
3,2001,Nevada,2.4,NaN
4,2002,Nevada,2.9,NaN
5,2003,Nevada,3.2,NaN


In [52]:
frame2.loc[1]

year     2001
state    Ohio
pop       1.7
debt      NaN
Name: 1, dtype: object

In [53]:
frame2.iloc[2]

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: 2, dtype: object

## 创建或者修改列，可以将空的列debt赋值为标量值或值数组。为不存在的列赋值会创建一个新的列。    `frame2["column"] = value/list/Series`<br>将列表或数组分配给列时，值的长度必须与 DataFrame 的长度匹配。如果你分配一个 Series，它的标签将完全重新对齐到 DataFrame 的索引，并在任何不存在的索引值中插入缺失值。<br>通过索引方式从DataFrame返回的列只是底层数据的视图而已，并不是副本。因此，对返回的Series所做的任何就地修改全都会反映到DataFrame上。应当通过Series的copy方法来复制列。

In [54]:
frame2

,year,state,pop,debt
0,2000,Ohio,1.5,NaN
1,2001,Ohio,1.7,NaN
2,2002,Ohio,3.6,NaN
3,2001,Nevada,2.4,NaN
4,2002,Nevada,2.9,NaN
5,2003,Nevada,3.2,NaN


In [55]:
frame2["debt"] = 16.5

In [56]:
frame2

,year,state,pop,debt
0,2000,Ohio,1.5,16.5
1,2001,Ohio,1.7,16.5
2,2002,Ohio,3.6,16.5
3,2001,Nevada,2.4,16.5
4,2002,Nevada,2.9,16.5
5,2003,Nevada,3.2,16.5


In [57]:
frame2["debt"] = np.arange(6.)

In [58]:
frame2

,year,state,pop,debt
0,2000,Ohio,1.5,0.0
1,2001,Ohio,1.7,1.0
2,2002,Ohio,3.6,2.0
3,2001,Nevada,2.4,3.0
4,2002,Nevada,2.9,4.0
5,2003,Nevada,3.2,5.0


In [59]:
val = pd.Series([-1.2, -1.5, -1.7], index=[2, 4, 5])

In [60]:
frame2["debt"] = val

In [61]:
frame2

,year,state,pop,debt
0,2000,Ohio,1.5,NaN
1,2001,Ohio,1.7,NaN
2,2002,Ohio,3.6,-1.2
3,2001,Nevada,2.4,NaN
4,2002,Nevada,2.9,-1.5
5,2003,Nevada,3.2,-1.7


### 删除列，关键字`del`可以像在字典中那样删除列。<br>`del frame2["column"]`

In [62]:
frame2["eastern"] = frame2["state"] == "Ohio"

In [63]:
frame2

,year,state,pop,debt,eastern
0,2000,Ohio,1.5,NaN,True
1,2001,Ohio,1.7,NaN,True
2,2002,Ohio,3.6,-1.2,True
3,2001,Nevada,2.4,NaN,False
4,2002,Nevada,2.9,-1.5,False
5,2003,Nevada,3.2,-1.7,False


In [64]:
del frame2["eastern"]

In [65]:
frame2

,year,state,pop,debt
0,2000,Ohio,1.5,NaN
1,2001,Ohio,1.7,NaN
2,2002,Ohio,3.6,-1.2
3,2001,Nevada,2.4,NaN
4,2002,Nevada,2.9,-1.5
5,2003,Nevada,3.2,-1.7


## 通过索引方式从DataFrame返回的列只是底层数据的视图而已，并不是副本。因此，对返回的Series所做的任何就地修改全都会反映到DataFrame上。应当通过Series的`copy`方法来复制列。

In [66]:
frame2['pop']=10

In [67]:
frame2

,year,state,pop,debt
0,2000,Ohio,10,NaN
1,2001,Ohio,10,NaN
2,2002,Ohio,10,-1.2
3,2001,Nevada,10,NaN
4,2002,Nevada,10,-1.5
5,2003,Nevada,10,-1.7


In [68]:
a = frame2['debt'].copy()

In [69]:
a

0    NaN
1    NaN
2   -1.2
3    NaN
4   -1.5
5   -1.7
Name: debt, dtype: float64

In [70]:
a * 2

0    NaN
1    NaN
2   -2.4
3    NaN
4   -3.0
5   -3.4
Name: debt, dtype: float64

In [71]:
frame2

,year,state,pop,debt
0,2000,Ohio,10,NaN
1,2001,Ohio,10,NaN
2,2002,Ohio,10,-1.2
3,2001,Nevada,10,NaN
4,2002,Nevada,10,-1.5
5,2003,Nevada,10,-1.7


### 嵌套字典的字典，如果将嵌套字典传给DataFrame，pandas就会将外层字典的键解释为列，将内层字典的键解释为行索引。

In [72]:
populations = {"Ohio": {2000: 1.5, 2001: 1.7, 2002: 3.6},"Nevada": {2001: 2.4, 2002: 2.9}}

In [73]:
frame3 = pd.DataFrame(populations)

In [74]:
frame3

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


## 转置 DataFrame（交换行和列）    `frame3.T`
注意，如果列没有所有相同的数据类型，转置则会丢弃列的数据类型，因此转置以及再次转置返回原先的矩阵会导致丢失先前的类型信息。

In [75]:
frame3.T

,2000,2001,2002
Ohio,1.5,1.7,3.6
Nevada,NaN,2.4,2.9


## 明确指定了索引，改变内层字典键的顺序。

In [76]:
pd.DataFrame(populations, index=[2001, 2002, 2003])

,Ohio,Nevada
2001,1.7,2.4
2002,3.6,2.9
2003,NaN,NaN


### 由Series组成的字典，

In [77]:
frame3

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


In [78]:
pdata = {"Ohio": frame3["Ohio"][:-1],"Nevada": frame3["Nevada"][:2]}

In [79]:
pd.DataFrame(pdata)

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4


## DataFrame构造函数所能接收的各种数据如下表

| 类型 | 说明 |
|-------|-------|
| 二维ndarray | 数据矩阵，传入可选的行标签和列标签 |
| 由数组列表或元组组成的字典 | 每个序列会变成 DataFrame的一列，所有序列的长度必须相同 |
| NumPy的结构化/记录化数组 | 处理方式与“由数组组成的字典”一致 |
| 由 Series 组成的字典 | 每个值成为一列。如果没有显式指定索引，则各Series的索引会被合并成结果的行索引 |
| 由字典组成的字典 | 各内部字典成为一列，键会被合并成结果的行索引，处理方式与由 Series 组成的字典一致 |
| 字典或 Series 的列表 | 各项会成为DataFrame的一行字典键或Series索引合并后成为 DataFrame的列标签 |
| 由列表或元组组成的列表 | 处理方式与“二维ndarray”一致 |
| 另一个DataFrame | 该DataFrame的索引将会被沿用，除非显式指定了其他索引 |
| NumPy的MaskedArray | 处理方式与“二维ndarray”一致，只是在DataFrame结果中缺少掩码值 |

In [80]:
data = np.arange(9).reshape((3,3))

In [81]:
pd.DataFrame(data, index=["a", "b", "c"], columns=["l1", "l2", "l3"])

,l1,l2,l3
a,0,1,2
b,3,4,5
c,6,7,8


In [82]:
date1 = {'l1':[1, 2, 3], 'l2':[4, 5, 6], 'l3':[7, 8, 9]}
pd.DataFrame(date1)

,l1,l2,l3
0,1,4,7
1,2,5,8
2,3,6,9


In [83]:
date2 = {'l1':{'k1':1, 'k2':2, 'k3':3},'l2':{'k1':4, 'k2':5, 'k3':6}}
pd.DataFrame(date2)

,l1,l2
k1,1,4
k2,2,5
k3,3,6


## 设置DataFrame的`index`和`columns`的`name`属性。

In [84]:
frame3

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


In [85]:
frame3.index.name = "year"

In [86]:
frame3.columns.name = "state"

In [87]:
frame3

state,Ohio,Nevada
year,,
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


## 将 DataFrame 中包含的数据作为二维 ndarray 返回。    `frame3.to_numpy()`<br>如果 DataFrame 的列是不同的数据类型，则将选择返回数组的数据类型以容纳所有列。

In [88]:
frame3.to_numpy()

array([[1.5, nan],
       [1.7, 2.4],
       [3.6, 2.9]])

In [89]:
frame2

,year,state,pop,debt
0,2000,Ohio,10,NaN
1,2001,Ohio,10,NaN
2,2002,Ohio,10,-1.2
3,2001,Nevada,10,NaN
4,2002,Nevada,10,-1.5
5,2003,Nevada,10,-1.7


In [90]:
frame2.to_numpy()

array([[2000, 'Ohio', 10, nan],
       [2001, 'Ohio', 10, nan],
       [2002, 'Ohio', 10, -1.2],
       [2001, 'Nevada', 10, nan],
       [2002, 'Nevada', 10, -1.5],
       [2003, 'Nevada', 10, -1.7]], dtype=object)

## pandas的索引对象负责存储轴标签（包括DataFrame的列名）和其他元数据（比如轴名称或标签）。
Index对象是不可变的，因此无法修改。  
不可变性可以使索引对象在多个数据结构之间安全共享。  
除了类似于数组，索引也类似于一个大小固定的集合  
与Python集合不同，pandas的索引可以包含重复的标签，选择重复的标签，会选取所有对应的结果。

In [91]:
obj = pd.Series(np.arange(3), index=["a", "b", "c"])

In [92]:
obj

a    0
b    1
c    2
dtype: int64

In [93]:
index = obj.index

In [94]:
index

Index(['a', 'b', 'c'], dtype='object')

In [95]:
index[1:]

Index(['b', 'c'], dtype='object')

In [117]:
index[1] = "d"

TypeError: Index does not support mutable operations

In [97]:
labels = pd.Index(np.arange(3))

In [98]:
labels

Index([0, 1, 2], dtype='int64')

In [99]:
obj2 = pd.Series([1.5, -2.5, 0], index=labels)

In [100]:
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [101]:
obj2.index is labels

True

In [102]:
frame3

state,Ohio,Nevada
year,,
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


In [103]:
frame3.columns

Index(['Ohio', 'Nevada'], dtype='object', name='state')

In [104]:
"Ohio" in frame3.columns

True

In [105]:
2003 in frame3.index

False

In [106]:
pd.Index(["foo", "foo", "bar", "bar"])

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

## 每个索引都有一些集合逻辑的方法和属性，可用于处理索引所包含数据的常见问题。下表的一些方法和属性。

| 方法 | 说明 |
|-------|-------|
| append() | 连接另一个索引对象，产生一个新的索引 |
| difference() | 计算差集，并得到一个索引 |
| intersection() | 计算交集 |
| union() | 计算并集 |
| isin() | 计算一个指示值是否都包含在参数集合中的布尔型数组 |
| delete() | 删除索引i处的元素，并得到新的索引 |
| drop() | 删除传入的值，并得到新的索引 |
| insert() | 将元素插入索引i处，并得到新的索引 |
| is_monotonic | 当各元素均大于等于前一个元素时，返回True |
| is_unique | 当索引没有重复值时，返回 True |
| unique() | 计算索引中唯一值的数组 |

In [107]:
i1 = pd.Index([1, 2, 3])

In [108]:
i2 = pd.Index([3, 4, 5])

In [109]:
i3 = i1.append(i2)

In [110]:
i3

Index([1, 2, 3, 3, 4, 5], dtype='int64')

In [111]:
i4 = i1.difference(i2)

In [112]:
i4

Index([1, 2], dtype='int64')

In [113]:
i1.intersection(i2)	

Index([3], dtype='int64')

In [114]:
i1.union(i2)

Index([1, 2, 3, 4, 5], dtype='int64')

In [115]:
i1.isin(i2)

array([False, False,  True])

In [116]:
i1.drop(3)

Index([1, 2], dtype='int64')

In [121]:
frame3

state,Ohio,Nevada
year,,
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9


In [122]:
frame3

state,Ohio,Nevada
year,,
2000,1.5,NaN
2001,1.7,2.4
2002,3.6,2.9
